Carlos Bravo Garrán - 100474964

# __Seed Clustering__

In [ ]:

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot') or plt.style.use('ggplot')

from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans


import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

### __1. Load the dataset__

Load the seed dataset from a CSV file. 

The features are stored in `X`, and the target class labels are stored in `y`.

Add seed variable for random states (`100474964`)


In [ ]:
data = pd.read_csv('data/semillas.csv')
X = data.drop(columns=['clase'])
y = data['clase']

seed = 100474964

print(data.head())

### __2. Comparison of Scalers__

This section identifies the most appropriate scaler for the seed dataset before applying clustering algorithms. Scaling ensures that all features contribute equally to the distance calculations.

Three scalers are compared:
- MinMaxScaler
- RobustScaler
- StandardScaler

The scaled data is projected into 2D using PCA for visual evaluation.

In [ ]:
scalers = {
    'MinMaxScaler': MinMaxScaler(),
    'RobustScaler': RobustScaler(),
    'StandardScaler': StandardScaler()
}

plt.figure(figsize=(18, 5))

for i, (name, scaler) in enumerate(scalers.items(), 1):
    pipeline = make_pipeline(scaler, PCA(n_components=2, random_state=seed))
    
    X_pca = pipeline.fit_transform(X)
    
    plt.subplot(1, 3, i)
    scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y, cmap='viridis', alpha=0.7, s=50)
    plt.title(f'{name} + PCA')
    plt.xlabel('PC1')
    plt.ylabel('PC2')
    plt.grid(True)

plt.suptitle('Comparison of Scalers with PCA (By Seed Class)', fontsize=16)
plt.tight_layout()
plt.show()


#### 2.1 Best Scaler Selection

After observing the PCA plots:

- **MinMaxScaler**: The data points are well distributed with moderate separation between different seed classes. Although some overlap exists, the distribution appears balanced and suitable for clustering.
- **RobustScaler**: There is a good separation of classes, but the spread of the data is larger, which may not be ideal for density-based methods.
- **StandardScaler**: While resistant to outliers, the classes are not well separated, making clustering more difficult.

The scaler selected is **MinMaxScaler** because it provides a balanced and homogeneous distribution of the data, facilitating the identification of clusters without introducing large variations in scale.


#### 2.2 Variance Explained by PCA

To ensure that the 2D projection using PCA retains enough information from the original dataset, calculate, after applying each scaler, the variance explained by the two principal components and the total variance.


In [ ]:
variance_ratios = {}

for name, scaler in scalers.items():
    X_scaled = scaler.fit_transform(X)
    pca = PCA(n_components=2, random_state=seed)
    X_pca = pca.fit_transform(X_scaled)
    
    variance_explained = np.sum(pca.explained_variance_ratio_)
    variance_ratios[name] = {
        'PC1': pca.explained_variance_ratio_[0],
        'PC2': pca.explained_variance_ratio_[1],
        'Total': variance_explained
    }

variance_table = pd.DataFrame.from_dict(variance_ratios, orient='index')
variance_table.index.name = 'Scaler'
variance_table.reset_index(inplace=True)

display(variance_table)


The results show that all three scalers achieve a high variance explanation (>85%), indicating that the 2D PCA projection is representative of the original dataset in every case.

Among them, **MinMaxScaler** achieves the highest variance explained, with 91.81%.

It's confirmed that using **MinMaxScaler** is appropriate, as it retains the largest proportion of the original data variance after dimensionality reduction. Therefore, MinMaxScaler is selected for the clustering tasks.

#### 2.3 Scaling and PCA
The dataset was scaled using the selecter scaler (**MinMaxScaler**) and reduced to two dimensions using PCA

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline

pipeline = make_pipeline(
    MinMaxScaler(),
    PCA(n_components=2, random_state=seed)
)

X_final_pca = pipeline.fit_transform(X)

plt.figure(figsize=(4, 3))
plt.scatter(X_final_pca[:, 0], X_final_pca[:, 1], c=y, cmap='viridis', alpha=0.7)
plt.title('MinMaxScaler + PCA (Final Preprocessing)')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('MinMaxScaler + PCA', fontsize=10)
plt.grid(True)
plt.show()


### __3. Clustering__

Apply unsupervised clustering techniques on the PCA-transformed seed dataset to identify natural groupings:

- K-Means
- Hierarchical Clustering
- DBSCAN

For each method,the key hyperparameters are tuned, the results visualized and their performance compared to determine which best captures the structure of the data.


#### __3.1 K-means clustering__

Based on the visual inspection of the PCA plot, k = 3 is initially selected as a reasonable number of clusters.

##### __Model creation and training__

In [ ]:
modelo_kmeans = KMeans(
    n_clusters=3,
    n_init=25,
    random_state=seed
)

modelo_kmeans.fit(X_final_pca)


##### __Cluster prediction__

In [ ]:
y_pred = modelo_kmeans.predict(X_final_pca)


##### __Cluster Visualization and Evaluation__


In [ ]:
colors = ['yellow', 'purple', 'green', 'orange', 'purple']

fig, ax = plt.subplots(1, 1, figsize=(6, 5))

for cluster in np.unique(y_pred):
    ax.scatter(
        x = X_final_pca[y_pred == cluster, 0],
        y = X_final_pca[y_pred == cluster, 1],
        color = colors[cluster],
        label = f"Cluster {cluster}",
        edgecolor = 'black',
        marker = 'o'
    )

ax.scatter(
    x = modelo_kmeans.cluster_centers_[:, 0],
    y = modelo_kmeans.cluster_centers_[:, 1],
    c = 'red',
    s = 200,
    marker = '*',
    label = 'Centroids'
)

ax.set_title('Clusters generated by KMeans (K=3)')
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.legend()
ax.grid(True)
plt.show()


The K-Means clustering with k = 3 produced three clearly defined groups in the PCA-transformed space.

- Each cluster is compact and well-separated from the others, with minimal overlap between groups.
- The centroids are located close to the center of each cluster, indicating that the algorithm has correctly captured the dense regions of the data.
- The overall structure suggests that the data naturally forms three main groups, supporting the choice of k = 3.
- Although the clusters are not perfectly spherical, the distribution around each centroid is balanced and coherent.

In conclusion, the visual inspection confirms that K-Means with k = 3 is a good fit for the structure observed in the seed dataset.

Even so, other values of k will be tested to ensure robustness.

#### __Evaluation of K-Means with Different Values of k__

In this section, the performance of the K-Means algorithm is evaluated using different values of k (number of clusters). This helps identify the optimal number of clusters that best represents the structure of the data.

Plots are generated for k = 2 and k = 4, comparing the distribution of data in the PCA space.


In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 5))

# Results for K = 2
y_predict_2 = KMeans(n_clusters=2, n_init=25, random_state=seed).fit_predict(X=X_final_pca)
ax[0].scatter(
    x = X_final_pca[:, 0],
    y = X_final_pca[:, 1],
    c = y_predict_2,
    marker = 'o',
    edgecolor = 'black'
)
ax[0].set_title('KMeans with K=2')
ax[0].set_xlabel('PC1')
ax[0].set_ylabel('PC2')
ax[0].grid(True)

# Results for K = 4
y_predict_4 = KMeans(n_clusters=4, n_init=25, random_state=seed).fit_predict(X=X_final_pca)
ax[1].scatter(
    x = X_final_pca[:, 0],
    y = X_final_pca[:, 1],
    c = y_predict_4,
    marker = 'o',
    edgecolor = 'black'
)
ax[1].set_title('KMeans with K=4')
ax[1].set_xlabel('PC1')
ax[1].set_ylabel('PC2')
ax[1].grid(True)

plt.tight_layout()
plt.show()



- With **k = 2**, the data is divided into two large groups, losing important internal structures.
- With **k = 4**, the clustering tends to oversegment the data, creating small clusters that do not correspond to natural groupings.
- With **k = 3**, the data is divided into three compact and well-separated groups, without significant overlap or oversegmentation.

The visual comparison confirms that **k = 3** is the most appropriate number of clusters for this dataset.